In [20]:
from dotenv import load_dotenv
import os
from glob import glob

## 벡터저장소 로드

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

#OpenAI 임베딩 모델(text-embedding-3-small)을 준비
embeddings = OpenAIEmbeddings(
    model = 'text-embedding-3-small'
)

#Chroma 벡터DB를 기존 저장소(chroma_db)에서 불러옴
vectorstore = Chroma(    
    embedding_function=embeddings,
    collection_name="chroma_test",
    persist_directory='./chroma_db'
)

print(f'벡터 저장소에 저장된 문서 수 : {vectorstore._collection.count()}')

벡터 저장소에 저장된 문서 수 : 22


## 프롬프트 템플릿 및 LLM 준비

In [ ]:
#다중 메세지 전송
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

#OpenAI LLM(gpt-4o-mini) 인스턴스 준비
llm = ChatOpenAI (
    model='gpt-4o-mini',
    temperature=0.3,
    max_tokens = 100 
)


#시스템/유저 역할로 구성된 메시지 프롬프트 템플릿 생성
messages = [
    ("system", "You are a helpful assistant."),
    ("user", "{query}")
]

prompt = ChatPromptTemplate.from_messages(messages)

print(prompt)

input_variables=['query'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))]


In [ ]:
#프롬프트 템플릿의 입력 변수 확인
print(prompt.input_variables)

['query']


In [ ]:
#실제질의로 프롬프트 생성 및 출력력
prompt_text = prompt.format_prompt(query='테슬라 창업자는 누구인가요?')
print(prompt_text)

messages=[SystemMessage(content='You are a helpful assistant.'), HumanMessage(content='테슬라 창업자는 누구인가요?')]


In [ ]:
#프롬프트를 LLM에 직접 전달하여 응답 출력력
response = llm.invoke(prompt_text)
print(response.content)

테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하드(Martin Eberhard), 마크 타페닝(Mark Tarpenning), 제프 스프링어(Jeffrey Brian Straubel)입니다. 테슬라는 2003년에 설립되었으며, 엘론 머스크는 2004년에 투자자로 참여한 후 CEO로 취임하여 회사의 발전에 큰 영향을 미쳤습니다. 이후


In [ ]:
# LCEL 체인 구성 : 프롬프트와 LLM을 파이프라인(체인)으로 연결
chain = prompt | llm

#체인의 입력 스키마 확인
print(chain)


first=ChatPromptTemplate(input_variables=['query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))]) last=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000022FD2555E90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000022FD24D3610>, root_client=<openai.OpenAI object at 0x0000022FD1A7B810>, root_async_client=<openai.AsyncOpenAI object at 0x0000022FD2555190>, model_name='gpt-4o-mini', temperature=0.3, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=100)


In [26]:
from pprint import pprint
pprint(chain.input_schema.schema())


{'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'PromptInput',
 'type': 'object'}


In [ ]:
#체인에 질의 전달 후 응답 출력
response = chain.invoke({"query": '테슬라 창업자는 누구인가요?'})
print(response.content)

테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하르드(Martin Eberhard), 제프리 브라이언(Jeffrey Brian), 이안 라이트(Ian Wright), 그리고 JB 스트라우벨(JB Straubel)입니다. 테슬라는 2003년에 설립되었으며, 엘론 머스크는 2004년에 투자자로 참여한 후 CEO로 취임하여 회사를 이끌어


### Prompt + LLM + Output Parser
#### StrOutputParser
LLM 응답을 문자열로 변환하는 파서(StrOutputParser)

In [28]:
response

AIMessage(content='테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하르드(Martin Eberhard), 제프리 브라이언(Jeffrey Brian), 이안 라이트(Ian Wright), 그리고 JB 스트라우벨(JB Straubel)입니다. 테슬라는 2003년에 설립되었으며, 엘론 머스크는 2004년에 투자자로 참여한 후 CEO로 취임하여 회사를 이끌어', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 27, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'finish_reason': 'length', 'logprobs': None}, id='run-06737b0a-50f3-4134-82f7-d0d6144b06da-0', usage_metadata={'input_tokens': 27, 'output_tokens': 100, 'total_tokens': 127})

In [33]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
output_parser.invoke(response)

'테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하르드(Martin Eberhard), 제프리 브라이언(Jeffrey Brian), 이안 라이트(Ian Wright), 그리고 JB 스트라우벨(JB Straubel)입니다. 테슬라는 2003년에 설립되었으며, 엘론 머스크는 2004년에 투자자로 참여한 후 CEO로 취임하여 회사를 이끌어'

In [ ]:
#프롬프트 → LLM → 파서로 이어지는 체인 구성
str_chain = prompt | llm | output_parser

query = '리비안의 설립년도는 언제인가요?'
str_response = str_chain.invoke(query)
print(str_response)

리비안(Rivian)은 2009년에 설립되었습니다. 이 회사는 전기차를 개발하는 미국의 자동차 제조업체로, 특히 전기 픽업트럭과 SUV에 주력하고 있습니다.


##### JsonOutputParser - Json 출력

In [41]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()

json_response = chain.invoke('테슬라 창업자는 누구인가요? JSON 형식으로 올려주세요.')
print(json_response)
print('==============')

json_parser_output = json_parser.invoke(json_response)
print(json_parser_output)


content='```json\n{\n  "회사": "테슬라",\n  "창립자": [\n    {\n      "이름": "마틴 에버하드",\n      "역할": "공동 창립자"\n    },\n    {\n      "이름": "마크 타페닝",\n      "역할": "공동 창립자"\n    },\n    {\n      "이름": "엘론 머스크",\n      "역할": "주요 투자자' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 35, 'total_tokens': 135, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'length', 'logprobs': None} id='run-362566e4-0b13-4a42-9260-ca79952c7b70-0' usage_metadata={'input_tokens': 35, 'output_tokens': 100, 'total_tokens': 135}
{'회사': '테슬라', '창립자': [{'이름': '마틴 에버하드', '역할': '공동 창립자'}, {'이름': '마크 타페닝', '역할': '공동 창립자'}, {'이름': '엘론 머스크', '역할': '주요 투자자'}]}


##### PydanticOutParser

In [47]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

#PydanticOutParser를 Person 객체와 연결결
class Person(BaseModel):
    name: str = Field(description="The name of the person")
    title: str = Field(description="The title or position of the person")

pydantic_parser = PydanticOutputParser(pydantic_object=Person)
#print('Pydantic Output Parser prompt:')
#print(pydantic_parser.get_format_instructions())
#print('=======================')

prompt = ChatPromptTemplate.from_messages([
    ('system', "Answer the user query. Wrap the output in 'json' tags\n{format_instructions}"),
    ('user', "{query}"),
]).partial(format_instructions=pydantic_parser.get_format_instructions())

In [48]:
person_chain = prompt | llm | pydantic_parser

response = person_chain.invoke({"query": "테슬라 창업자는 누구인가요?"})
response

Person(name='엘론 머스크', title='CEO 및 창립자')

## 출력방식

### Stream

In [50]:
import time

for chunk in llm.stream('테슬라 창업자는 누구인가요?'):
    print(chunk.content, end='', flush=True)
    time.sleep(0.1)

테슬라의 창립자는 엘론 머스크(Elon Musk)가 아닙니다. 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 이후 엘론 머스크가 2004년에 투자자로 참여하게 되고, 이후 CEO로 취임하면서 회사의 비전을 주도하게 되었습니다. 엘론 머스크는 테슬라

### Batch

In [53]:
questions = [
    "테슬라 창업자는 누구인가요?",
    "리비안의 설립년도는 언제인가요?"
]

responses = llm.batch(questions)

for response in responses:
    response.pretty_print()
    print()

================================== Ai Message ==================================

테슬라의 창립자는 엘론 머스크(Elon Musk)가 아닙니다. 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 테슬라에 투자하고 이후 CEO로 취임하면서 회사의 주요 인물로 자리잡았습니다. 따라서 엘론 머스크는 테슬라

================================== Ai Message ==================================

리비안(Rivian)은 2009년에 설립되었습니다. 이 회사는 전기차를 개발하고 생산하는 데 주력하고 있으며, 특히 전기 픽업트럭과 SUV 모델로 주목받고 있습니다.



## Runnable

In [ ]:
# 문서 검색기 생성
retriever = vectorstore.as_retriever(
    search_kwargs={"k":2}, # 가장 유사한 문서 2개 반환
)

query = '테슬라 창업자는 누구인가요?'
retrived_docs = retriever.invoke(query) # 쿼리에 대해 문서 검색
print(f"검색된 문서 수 : {len(retrived_docs)}")

retrived_docs_text = "\n".join([doc.page_content for doc in retrived_docs])
pprint(retrived_docs_text)
#검색된 문서 개수와 내용을 출력

검색된 문서 수 : 2
('테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.\n'
 '테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.')


In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel

#입력값을 그대로 반환하는 RunnablePassthrough를 병렬 실행 구조(RunnableParallel)로 감쌉니다

runnable = RunnableParallel(
    question=RunnablePassthrough()
)

runnable.invoke("테슬라 창업자는 누구인가요?")

#입력값("테슬라 창업자는 누구인가요?")을 그대로 question 키로 반환

{'question': '테슬라 창업자는 누구인가요?'}

In [ ]:
from langchain_core.runnables import RunnableLambda

def count_num_words(text):
    return len(text.split())

runnable = RunnableParallel( #병렬처리를 위함->빠른 성능
    question=RunnablePassthrough(), #있는 그대로의 모습 #입력값을 question 키로 그대로 반환
    word_count=RunnableLambda(count_num_words) #입력값을 함수로 가공 #입력값의 단어 수를 word_count 키로 반환
)

runnable.invoke("테슬라 창업자는 누구인가요?")


{'question': '테슬라 창업자는 누구인가요?', 'word_count': 3}

## Prompt 템플릿 생성

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = """
Answer the question based only on the following context.
Do not user any external knowledge.
If the answer is not in the context, say "I don't know."

[Context]
{context}

[Question]
{question}

[Answer]
"""

prompt = ChatPromptTemplate.from_template(template)

prompt.pretty_print() #프롬프트 템플릿의 내용을 보기 좋게 출력해주는 함수



================================ Human Message =================================


Answer the question based only on the following context.
Do not user any external knowledge.
If the answer is not in the context, say "I don't know."

[Context]
{context}

[Question]
{question}

[Answer]



In [ ]:
# 벡터 검색기 : 쿼리와 가장 유사한 문서 2개 반환
retriever = vectorstore.as_retriever(
    search_kwargs={"k":2}
)

# 문서 포맷터 함수 : 검색된 여러 docs의 page_content를 줄바꿈 2개로 이어 하나의 긴 문자열로 만듦듦
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# 체인 구성 : retriever의 결과를 format_docs에 넘기는 chain을 만듦
retriever_chain = retriever | format_docs

# 체인 실행 : 질문을 체인에 넣어서 실행
response = retriever_chain.invoke("테슬라 창업자는 누구인가요?")
pprint(response) # pretty-print의 약자

#이 전체 내용이 context에 들어가는 것임

('테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.\n'
 '\n'
 '테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.')


#### RAG Chain 연결

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature = 0,
    max_tokens = 100 # 최대 답변 토큰큰
    
)

# LLM 체인 생성
rag_chain = (
    {"context": retriever_chain, "question": RunnablePassthrough()} # context에는 retriever_chain을 전달, question에는 RunnablePassthrough(입력값 그대로) 전달
    | prompt # 위 두값을 prompt에 넣고
    | llm # llm에 전달
    | StrOutputParser() #llm의 응답을 stroutputparser 문자열로 변환환
)

# RAG 체인 실행
response = rag_chain.invoke("테슬라 창업자는 누구인가요?") #질문을 체인에 넣어서 실행행
print(response)

테슬라의 창업자는 마틴 에버하드와 마크 타페닝입니다.


In [74]:
import gradio as gr

def answer_invoke(message, history):
    response = rag_chain.invoke(message)
    return response

demo = gr.ChatInterface(fn=answer_invoke, title='QA Bot')

demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


c:\Users\KOSALOVE\AppData\Local\pypoetry\Cache\virtualenvs\langchain-env-I6Y7WdhY-py3.11\Lib\site-packages\gradio\analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


In [75]:
demo.close()

Closing server running on port: 7861
